In [28]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato,off_Reliance
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
from funcoes import CalculosRiscoPort

In [29]:
#Chama as classes do databases e as classes do funções data
rel_off_p = off_Reliance('Primeny')
rel_off_c = off_Reliance('Cubo')
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()
calc = CalculosRiscoPort(offshore = True)

In [30]:
#Procura os parametros para rodar as funções

#Puxando a ultima data do mes anterior
data = bds.banco.hoje()
data = datetime.datetime.date(data)
hoje = fdt.workday(data, n_dias=-4)

#Traz os de_paras de mandatos (mandato anterior para o atual)
mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_mandato')
mandatos = mandatos[['Anterior','Novo']]
#mandatos = dict(mandatos.values)
    
# #Traz os de_paras de classificação gps para asset Allocation
dicio_classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacoes=dicio_classificacoes[['Classificação GPS','De Para - Asset Allocation']]
dicio_classificacoes = dict(dicio_classificacoes.values)

#Novos mandatos
novos_mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Novos_mandatos')
novos_mandatos= novos_mandatos[['Portfolio','Classe','Tático']]

#Trazer os produtos GPS e segregar o que é imobiliario (caso precise segregar o que é Reits / ILiquidos)
produtos_gps = crm.produtos_off()
produtos_gps =produtos_gps[['name','new_idmysql','new_tickerid','new_isin','new_idbds','new_idsistemaorigem','new_indgroupaxysidname']].rename(columns={'name':'Ativo','new_idmysql':'IdMysqlProduto','new_tickerid':'Ticker','new_isin':'ISIN','new_idsistemaorigem':'CodigoProduto'}).reset_index()
produtos_gps['ISIN'] = produtos_gps['ISIN'].str.lower()

#Trazendo o De Para de ativos e separando os dataframes para unir com a carteira
de_para_ativos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Tb De Para Subclass')

# #Trazendo todos os produtos da base Reliance
# produtos_rel= rel_off_p.produtos_off().drop(columns=['subclasse'])
# produtos_rel['ISIN'] = produtos_rel['ISIN'].str.lower()

#Trazendo o dicionario da classificação dos ativos dado pela area de investimentos (produtos reliance)
dicio_classificacao_investimentos_rel = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacao_investimentos_rel=dicio_classificacao_investimentos_rel[['Classificação Reliance-Investimentos','Classificação GPS - v1']]
dicio_classificacao_investimentos_rel = dict(dicio_classificacao_investimentos_rel.values)

#Dicionario Reliance Classe + Subclasse + Categoria
dicio_classe_sub_categoria = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_sub_categoria=dicio_classe_sub_categoria[['Classe_Subclasse_Categoria-Sistema','Classificação GPS - v2']]
dicio_classe_sub_categoria = dict(dicio_classe_sub_categoria.values)

#Saber a qual classe pertence cada subclasse
dicio_classe_subclasse_gps = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_subclasse_gps=dicio_classe_subclasse_gps[['Subclasse GPS','Classe']]
dicio_classe_subclasse_gps = dict(dicio_classe_subclasse_gps.values)

#Benchmarks
benchs = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
benchs=benchs[['De Para - Asset Allocation','Index_Bawm']]

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [31]:
#Pegando o primeiro dia de cada mes
data = fdt.hoje()
data = data - relativedelta(days=30)
data = date(data.year, data.month, 1) 
data=pd.to_datetime(data)

#Carteira do ultimo extrato
alocacao = rel_off_c.carteira_por_peso(data)
alocacao['NomeClasse']=alocacao['NomeClasse'].str.lower()

#dicionario de classes - Deixando tudo minusculo e equiparando com as classes de asset allocation
dicio_classes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classes=dicio_classes[['Classe_Alocacao_sistema','Classe GPS']]
dicio_classes = dict(dicio_classes.values)

del dicio_classes[np.nan]
dicio_classes = dict((k.lower(), v.lower()) for k, v in dicio_classes.items()) 
alocacao['NomeClasse']=alocacao['NomeClasse'].apply(lambda x:dicio_classes[x] )
alocacao=alocacao.dropna(subset=['nome_reduzido'])


del dicio_classificacoes[np.nan]
dicio_classificacoes = dict((k.lower(), v.lower()) for k, v in dicio_classificacoes.items()) 
alocacao['NomeClasse']=alocacao['NomeClasse'].apply(lambda x:dicio_classificacoes[x])

#Fazendo o merge com todos os possíveis portfolios
novos_mandatos['Classe'] = novos_mandatos['Classe'].str.lower()
novos_mandatos = novos_mandatos[novos_mandatos['Classe']!='alternativos long short asia']
novos_mandatos = novos_mandatos[novos_mandatos['Portfolio'].str.contains('MS') == False]

#Puxar os clientes para o for
clientes = alocacao['nome_reduzido'].drop_duplicates().to_list()
clientes = [x for x in clientes if x is not None]

#Puxar os portfolios para o for
portfolios = novos_mandatos['Portfolio'].drop_duplicates().to_list()

#dicionario de classes - Deixando tudo minusculo e equiparando com as classes de asset allocation
dicio_classes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classes=dicio_classes[['Classe_Alocacao_sistema','Classe GPS']]
dicio_classes = dict(dicio_classes.values)

#Rentabilidade das carteiras
rentabilidade = rel_off_c.rentabilidade(data)
rentabilidade = rentabilidade.loc[rentabilidade.groupby('nome_reduzido').DataProcessamentoFim.idxmin()].reset_index(drop=True)
rentabilidade = rentabilidade[['nome_reduzido','PontosRisco','RetornoLiquido_NoAno','RetornoLiquido_12Meses']]

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [32]:
pos = pd.DataFrame()
tracking= []
#clientes = ['REPAR_JB']

for cliente in clientes:
    nome=cliente
    df = alocacao[alocacao['nome_reduzido']==nome]
    df= pd.merge(left=df,right=novos_mandatos,left_on='NomeClasse',right_on='Classe',how='right')
    df = df[df['Classe'].str.contains('jbfo') == False]
    df['nome_reduzido']=df['nome_reduzido'].fillna(nome)
    df['PcInicioAno']= df['PcInicioAno'].fillna(0)
    df = df.rename(columns={'PcInicioAno':'Atual'})    
    df= df[['Portfolio','Classe','Tático','Atual']]
    
    for portfolio in portfolios:
        analise = df[df['Portfolio']==portfolio]
        analise = analise[['Classe','Tático','Atual']]
        analise['Classe']=analise['Classe'].str.lower()
        te=calc.tracking_error(analise)
        tracking.append({'Cliente': nome, 'portfolio': portfolio,'TE': te})
        
    
df = pd.DataFrame.from_dict(tracking)    
df['Mandato/TE']=df.apply(lambda row : str(row['portfolio']) +' / '+ str(round(row['TE'],2)),axis=1)
df = df.sort_values(['Cliente', 'TE'], ascending=[True, True]).reset_index(drop=True)


port_recomendado = df.loc[df.groupby('Cliente').TE.idxmin()].reset_index(drop=True)
port_recomendado=pd.merge(left=rentabilidade,right=port_recomendado, left_on='nome_reduzido',right_on='Cliente',how='left')
port_recomendado = port_recomendado[['Cliente','RetornoLiquido_NoAno','PontosRisco','portfolio','TE']].rename(columns={'RetornoLiquido_NoAno':'RetornoLiquido_2023','portfolio':'Mandato_Recomendado'})  
    

In [33]:
rank=[]
df['verifica']=df['Cliente']==df['Cliente'].shift(1)

for idx,row in df.iterrows():
    if row['verifica']==False:
        n=1
    else:
        n=n+1
    rank.append(n)    
            
df['Ranking']=rank    
df = df.groupby(['Cliente','Ranking'])['Mandato/TE'].aggregate('first').unstack().reset_index()
port_recomendado = pd.merge(left=port_recomendado,right=df,on='Cliente',how='left').sort_values(by=['Cliente']).dropna(subset=['Cliente'])

In [34]:
port_recomendado

,Cliente,RetornoLiquido_2023,PontosRisco,Mandato_Recomendado,TE,1,2,3,4,5,6,7
365,REPAR_JB,0.044888,323.644336,Core Portfolio 2,0.023789,Core Portfolio 2 / 0.02,Focus Portfolio 2 / 0.03,Core Blend / 0.03,Core Portfolio 3 / 0.03,Focus Portfolio 3 / 0.04,Core Portfolio 4 / 0.06,Focus Portfolio 4 / 0.06
